## Import *.pcap par tshark, labellisaton par Suricata, ML

### Set-Up

In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import collections
import seaborn as sns
from pprint import pprint
import os
import json

# import ipywidgets
# import warnings

# import pyshark
# import networkx as nx

# from sklearn.preprocessing import OrdinalEncoder, StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans, DBSCAN
# from sklearn.manifold import TSNE
# from sklearn.metrics import pairwise_distances, silhouette_score
# from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
# import umap

# from itertools import product

# PATH change to access library cyberlib
import sys
sys.path.append('/home/benjamin/Folders_Python/Cyber/libs')
import cyberlib as cbl

# to allow PyShark to run in Jupyter notebooks
# import nest_asyncio
# nest_asyncio.apply()

In [2]:
# logging set-up

lg = cbl.GetLogger('/home/benjamin/Folders_Python/Cyber/logs/pcap_labellisation.log')
logger = lg.get_custom_logger()

# start your engine
logger.info("-------- new run --------")

### Import *pcap by tshark, export to *.csv then DataFrame

In [3]:
# which *.pcap

DFNAME = 'smallFlows'

PCAPFILE = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/' + DFNAME + '.pcap'

In [4]:
# We use the tshark CLI to parse the *.pcap file and output a *.csv file for pandas
# doc here : https://www.wireshark.org/docs/man-pages/tshark.html

# tshark :
# -r                :   reads the *.pcap
# -2, -R "tcp"      :   does 2 passes and keeps packets part of TCP conversations only
# -T fields         :   outputs a file with fields
# -E header=y       :   keeps the fields names on first row for pd.read_csv
# -E separator=,    :   for *.csv format
# -e <fields>       :   desired output fields
# -o                :   formats of the data in the fields

cli="tshark -r " + PCAPFILE + """ -2 \
    -R "tcp" \
    -T fields -E header=y -E separator=, \
    -e _ws.col.Time -t ad \
    -e frame.number \
    -e eth.src -e eth.dst \
    -e ip.src_host -e ip.dst_host \
    -e ip.len -e ip.hdr_len -e ip.ttl \
    -e tcp.srcport -e tcp.dstport -e tcp.stream -e tcp.len \
    -e tcp.seq -e tcp.ack -e tcp.hdr_len -e tcp.time_relative \
    -e tcp.time_delta \
    -e tcp.flags \
    -o 'gui.column.format:"No","%m","Time","%t","Source","%s","Destination","%d","Protocol","%p","Length","%L","Info","%i"' \
    > ~/Folders_Python/Cyber/data/input_pcaps/to_csv/test.csv"""

%time exit_code = os.system(cli)

if exit_code == 0:
    logger.info('Executed successfully *.pcap to *.csv translation with tshark')
else:
    logger.error('Error while using tshark to translate from *.pcap to *.csv')
    raise NameError('Error while using tshark to translate from *.pcap to *.csv')

CPU times: user 1.2 ms, sys: 0 ns, total: 1.2 ms
Wall time: 2.04 s


In [5]:
filename = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/to_csv/test.csv'

with open(file=filename, encoding='utf-8') as f:
    df_raw = pd.read_csv(
        f,
        header=0,               # using first row as columns names. they are exported by tshark -E header=y
        on_bad_lines='warn'     # if a line does not have the right length, skip it but warn me
        )

In [6]:
df_raw

,_ws.col.Time,frame.number,eth.src,eth.dst,ip.src_host,ip.dst_host,ip.len,ip.hdr_len,ip.ttl,tcp.srcport,tcp.dstport,tcp.stream,tcp.len,tcp.seq,tcp.ack,tcp.hdr_len,tcp.time_relative,tcp.time_delta,tcp.flags
0,2011-01-25 19:52:22.484409,1,40:61:86:9a:f1:f5,00:1a:8c:15:f9:80,192.168.3.131,72.14.213.138,983,20,128,57011,80,0,943,1,1,20,0.000000,0.000000,0x0018
1,2011-01-25 19:52:22.514250,2,00:1a:8c:15:f9:80,40:61:86:9a:f1:f5,72.14.213.138,192.168.3.131,426,20,52,80,57011,0,386,1,944,20,0.029841,0.029841,0x0018
2,2011-01-25 19:52:22.708292,3,40:61:86:9a:f1:f5,00:1a:8c:15:f9:80,192.168.3.131,72.14.213.102,52,20,128,55950,80,1,0,0,0,32,0.000000,0.000000,0x0002
3,2011-01-25 19:52:22.713832,4,40:61:86:9a:f1:f5,00:1a:8c:15:f9:80,192.168.3.131,72.14.213.138,40,20,128,57011,80,0,0,944,387,20,0.229423,0.199582,0x0010
4,2011-01-25 19:52:22.727058,5,00:1a:8c:15:f9:80,40:61:86:9a:f1:f5,72.14.213.102,192.168.3.131,52,20,52,80,55950,1,0,0,1,32,0.018766,0.018766,0x0012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13703,2011-01-25 19:57:20.768701,13704,08:00:27:cc:3f:1b,52:54:00:12:35:02,10.0.2.15,65.55.15.244,40,20,128,2537,5480,407,0,5039,5738,20,71.195375,66.560501,0x0014
13704,2011-01-25 19:57:20.768769,13705,08:00:27:cc:3f:1b,52:54:00:12:35:02,10.0.2.15,207.46.105.186,40,20,128,2540,5480,409,0,398,93,20,70.606228,5.540471,0x0014
13705,2011-01-25 19:57:20.768861,13706,08:00:27:cc:3f:1b,52:54:00:12:35:02,10.0.2.15,96.17.8.49,40,20,128,2547,5480,419,0,496,8189,20,64.405045,64.259982,0x0014
13706,2011-01-25 19:57:20.768911,13707,08:00:27:cc:3f:1b,52:54:00:12:35:02,10.0.2.15,91.103.140.2,40,20,128,2546,5480,417,0,525,270,20,64.884164,64.357688,0x0014


In [7]:
# # ordinal encoding with Pandas

# columns_to_encode_as_ordinal = ['frame.number', 'eth.src', 'eth.dst', 'ip.src_host', 'ip.dst_host', 'tcp.flags']

# df_ord = pd.DataFrame()
# for c in columns_to_encode_as_ordinal:
#     codes, _ = pd.factorize(df_raw[c])
#     df_sup = pd.DataFrame(data={ c : list(codes) })
#     df_ord = pd.concat([df_ord, df_sup], axis=1)
    
# df = df_raw.drop(columns=columns_to_encode_as_ordinal)
# df.reset_index(drop=True)

# df = pd.concat([df, df_ord], axis=1)

# # columns_to_drop = ['TIMESTAMP_ts']
# # df.drop(columns=columns_to_drop, inplace=True)

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13708 entries, 0 to 13707
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _ws.col.Time       13708 non-null  object 
 1   frame.number       13708 non-null  int64  
 2   eth.src            13708 non-null  object 
 3   eth.dst            13708 non-null  object 
 4   ip.src_host        13708 non-null  object 
 5   ip.dst_host        13708 non-null  object 
 6   ip.len             13708 non-null  int64  
 7   ip.hdr_len         13708 non-null  int64  
 8   ip.ttl             13708 non-null  int64  
 9   tcp.srcport        13708 non-null  int64  
 10  tcp.dstport        13708 non-null  int64  
 11  tcp.stream         13708 non-null  int64  
 12  tcp.len            13708 non-null  int64  
 13  tcp.seq            13708 non-null  int64  
 14  tcp.ack            13708 non-null  int64  
 15  tcp.hdr_len        13708 non-null  int64  
 16  tcp.time_relative  137

### Labellisation by Suricata

In [9]:
DATA_OUTPUT = '/home/benjamin/Folders_Python/Cyber/data/outputs'

cli="suricata -r " + PCAPFILE + " -l " + DATA_OUTPUT + " -k none"

%time exit_code = os.system(cli)

if exit_code == 0:
    logger.info('Executed successfully *.pcap to EVE.json translation with suricata')
else:
    logger.error('Error while using suricata to analyse from *.pcap to EVE.json')
    raise NameError('Error while using suricata to analyse from *.pcap to EVE.json')

17/8/2023 -- 17:52:58 - <Notice> - This is Suricata version 6.0.10 RELEASE running in USER mode
17/8/2023 -- 17:52:58 - <Warning> - [ERRCODE: SC_ERR_NO_RULES(42)] - No rule files match the pattern /etc/suricata/rules/suricata.rules
17/8/2023 -- 17:52:58 - <Warning> - [ERRCODE: SC_ERR_NO_RULES_LOADED(43)] - 1 rule files specified, but no rules were loaded!
17/8/2023 -- 17:52:58 - <Warning> - [ERRCODE: SC_ERR_INITIALIZATION(45)] - Unix socket: UNIX socket bind(/var/run/suricata-command.socket) error: Address already in use
17/8/2023 -- 17:52:58 - <Warning> - [ERRCODE: SC_ERR_INITIALIZATION(45)] - Unable to create unix command socket
17/8/2023 -- 17:52:58 - <Notice> - all 5 packet processing threads, 4 management threads initialized, engine started.
17/8/2023 -- 17:52:59 - <Notice> - Signal Received.  Stopping engine.
17/8/2023 -- 17:52:59 - <Notice> - Pcap-file module read 1 files, 14261 packets, 9216531 bytes
CPU times: user 4.11 ms, sys: 13 µs, total: 4.12 ms
Wall time: 406 ms


In [10]:
# !suricata -r /home/benjamin/Folders_Python/Cyber/data/input_pcaps/test.pcap -l /home/benjamin/Folders_Python/Cyber/data/outputs -k none

In [11]:
# Pandas provides a useful method – json_normalize – for normalizing nested JSON fields into dataframe. Resulting columns use dot notation to signify nested objects, similar to how Elasticsearch does it

SURICATA_EVE_LOG = "/home/benjamin/Folders_Python/Cyber/data/outputs/eve.json"

with open (SURICATA_EVE_LOG) as packets:
    df_log = pd.json_normalize(
        [json.loads(packet) for packet in packets],
        max_level=1
    )

In [12]:
df_log

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,tls.subject,...,stats.decoder,stats.flow,stats.defrag,stats.flow_bypassed,stats.tcp,stats.detect,stats.app_layer,stats.http,stats.ftp,stats.file_store
0,2011-01-25T19:52:22.968559+0100,2.051404e+15,20.0,tls,192.168.3.131,52152.0,72.14.213.147,443.0,TCP,"C=US, ST=California, L=Mountain View, O=Google...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-25T19:52:23.466591+0100,7.151276e+14,100.0,http,192.168.3.131,55953.0,65.55.206.209,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-25T19:52:23.832607+0100,3.339836e+14,161.0,http,192.168.3.131,55959.0,65.55.5.231,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01-25T19:52:23.762574+0100,7.354749e+13,156.0,http,192.168.3.131,55956.0,66.235.139.121,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-01-25T19:52:24.330674+0100,7.238914e+14,186.0,http,192.168.3.131,55966.0,63.215.202.48,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876,2011-01-25T19:52:22.484409+0100,1.547619e+15,NaN,flow,192.168.3.131,56057.0,65.54.95.66,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3877,2011-01-25T19:52:22.484409+0100,1.125535e+15,NaN,flow,192.168.3.131,56136.0,206.108.207.163,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3878,2011-01-25T19:52:22.484409+0100,1.266440e+15,NaN,flow,192.168.3.131,56165.0,65.54.95.140,80.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3879,2011-01-25T19:52:22.484409+0100,7.035999e+14,NaN,flow,10.0.2.15,2532.0,207.46.125.254,7001.0,UDP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
len(df_log['pcap_cnt'].unique())

1209

In [14]:
df_log.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
timestamp,3881,1209,2011-01-25T19:52:22.484409+0100,1376,NaN,NaN,NaN,NaN,NaN,NaN,NaN
flow_id,3878.0,NaN,NaN,NaN,1133037968938332.5,677999154369971.5,3833734242459.0,505285657870996.75,1133124511906311.0,1765455366794224.0,2250187533138771.0
pcap_cnt,2483.0,NaN,NaN,NaN,6079.509062,3880.641344,20.0,2309.0,6643.0,9395.0,14235.0
event_type,3881,9,flow,1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
src_ip,3878,74,192.168.3.131,1866,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
stats.detect,3,3,"{'engines': [{'id': 0, 'last_reload': '2023-08...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stats.app_layer,3,2,"{'flow': {'http': 264, 'ftp': 0, 'smtp': 0, 't...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stats.http,3,1,"{'memuse': 0, 'memcap': 0}",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stats.ftp,3,1,"{'memuse': 0, 'memcap': 0}",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
